In [8]:
import requests
import json

In [83]:
class FlashTranslator:
    
    def __init__(self):
        self.kr = "http://36kr.com/api/info-flow/newsflash_columns/newsflashes?per_page=10"
        self.baidu = "http://translate.baidu.com/v2transapi"
        self.key_words = set(['created_at', 'published_at', 'updated_at', 'title', 'description', 'id'])
        self._kr_src = None
        self.result = None
        
    @property
    def kr_src(self):
        if self._kr_src is None:
            r = requests.get(self.kr)
            res = r.json()['data']['items']
            self._kr_src = [ {k: v for k,  v in row.items() if k in key_words} for row in res]
        return self._kr_src
    
    def translate(self):
        query = ""
        for d in self.kr_src:
            current = d['title'] + '\n' + d['description'].replace('\n', '') + '\n'
            query += current

        post_data = {
            'from': 'zh',
            'to': 'en',
            'query': query
        }

        p = requests.post(self.baidu, post_data)
        res = p.json()
        return res['trans_result']['data']
    
    def combine(self):
        data = self.translate()
        i = 0
        new_data = []
        for each_dict in self.kr_src:
            current = each_dict.copy()
            if current['title'].strip() == data[i]['src'].strip():
                current['translated_title'] = data[i]['dst']
            if current['description'].replace('\n', '').strip() == data[i+1]['src'].strip():
                current['translated_description'] = data[i+1]['dst']
            new_data.append(current)
            i += 2
        self.result = new_data

In [84]:
a = FlashTranslator()


a.combine()
a.result

[{'created_at': '2017-07-21 22:42:30',
  'description': '国外媒体今日发表文章，基于历史记录，苹果2014年9月9日星期二发布了iPhone 6和6 Plus，2015年9月9日星期三发布了iPhone 6s和6s Plus，2016年9月7日星期三发布了iPhone 7和7 Plus。基于此，iPhone 8、iPhone 7s和7s Plus最可能的发布日期是2017年9月5日星期二或9月6日星期三。（Sina Tech）',
  'id': 70757,
  'published_at': '2017-07-21 22:42:30',
  'title': '外媒：iPhone 8或于9月6日发布，最高售价1200美元',
  'translated_description': 'Foreign media published an article today, based on historical records, on Tuesday September 9, 2014 Apple released the iPhone 6 and Plus 6, released on Wednesday September 9, 2015 iPhone 6S and 6S Plus, released on Wednesday September 7, 2016 iPhone 7 and Plus 7. Based on this, the most likely release date for iPhone 8, iPhone, 7S, and 7S Plus is Tuesday, September 5, 2017, or Wednesday, September 6th. (Sina, Tech)',
  'translated_title': 'Foreign media: iPhone 8 or released in September 6th, the highest selling price of $1200',
  'updated_at': '2017-07-21 22:42:30'},
 {'created_at': '2017-07-21 19:43:28',
  'descrip

- The `id` is incremental

In [61]:
for x in data:
    print(x.get('id'))

70757
70755
70754
70753
70752
70750
70747
70746
70745
70744
